# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, qnormal, normal
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import os
import joblib
import shutil
import requests


In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'your experiment name here'

experiment=Experiment(ws, experiment_name)


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling(
    {
    "--max_depth": qnormal(6, 3, 4),
    "--gamma": choice(0.0, 0.1, 0.5, 1.0, 5.0),
    "--learning_rate": uniform(0.01, 0.2),
    "--alpha": uniform(0, 10),
    "--lambda": uniform(0, 10),    
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
os.makedirs("./training", exist_ok=True)
shutil.copy("train.py","./training")

# Create a SKLearn estimator for use with train.py
estimator = SKLearn(source_directory = './training',
              compute_target=compute_target,
              entry_script='train.py',
              pip_packages=["pyarrow>=0.12.0", "pyspark", "xgboost"])

primary_metric_name = "AUC_weighted"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     policy=None,
                                     max_total_runs=20,
                                     max_concurrent_runs=3,
                                     )

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_run_config, tag={"type": "hyperdrive_opt"})

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
hyperopt_run.wait_for_completion(show_output=True)
assert (hyperopt_run.get_status() == "Completed")
best_run = hyperopt_run.get_best_run_by_primary_metric()

In [ ]:
best_model = best_run.register_model(model_name="wine-quality-model", model_path="outputs/wine_hyperdrive.joblib")
print(best_model.properties)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
environment = Environment('my-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults==1.20.0',
    'inference-schema[numpy-support]==1.1.0',
    'joblib==1.0.0',
    'numpy==1.19.5',
    'scikit-learn=={}'.format(sklearn.__version__),
    'xgboost==1.3.1'
])
environment.save_to_directory('env_hyperdrive.yml')
service_name = 'wine-quality-classification'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[best_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
primary, _ = service.get_keys()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
data={"data":
         {
             "is_red_wine": 1,
             "fixed_acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000
         },
         {
             "is_red_wine": 0,
             "fixed_acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000
         }
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri,json_data, headers=header)
print(resp)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())
#service.delete()